In [1]:
import pandas as pd
import requests

In [2]:
query_user = "Find me the cheapest journey from London to Tokyo for one adult departing on the 30 August 2023 and returning on the 15 September 2023"

query = f'''Based on the user query: {query_user} do the following: 
Return all of the relevant information required to book the flights.
Some Journeys are indirect meaning they have multiple legs, remember to factor this into your calculation. The prices quoted are the total price for the journey. Use search to return links to the flight booking pages.
'''


In [3]:
# read keys
import yaml
import os

def read_config():
    # Get the directory of the current script
    # script_dir = os.path.dirname(os.path.realpath(__file__))
    script_dir = "../src/"

    # Construct the full path to the configuration file
    file_path = os.path.join(script_dir, "apikeys.yml")

    with open(file_path, 'r') as stream:
        try:
            configs = yaml.safe_load(stream)
            api_key = configs['amadeues_flights']['api_key']
            api_secret = configs['amadeues_flights']['api_secret']
            openai_key = configs['openai']['openai_key']
            hugging_api_key = configs['huggingfacehub']['hugging_api_key']
            return api_key, api_secret, openai_key, hugging_api_key
        except yaml.YAMLError as exc:
            print(exc)
            
    return api_key, api_secret, openai_key

class SingletonToken:
    __token = None

    @classmethod
    def set_token(cls, token):
        cls.__token = token

    @classmethod
    def get_token(cls):
        return cls.__token

api_key, api_secret, openai_key, hugging_api_key = read_config()

In [69]:
import numpy as np
def journey_data(response_flights_data, response_airline_lookup_data, originLocationCode, destinationLocationCode):
    # Load the data into a DataFrame
    df = pd.DataFrame(response_flights_data)
    df_airline_codes = pd.json_normalize(response_airline_lookup_data)
    
    # Extract itineraries, validatingAirlineCodes, price (total and currency) and id into separate dataframes
    df_itineraries = df[['id', 'itineraries']].explode('itineraries').reset_index(drop=True)
    
    # In the itineraries column, each cell is a dictionary. So, we need to convert those dictionaries into separate columns.
    df_itineraries = df_itineraries.join(pd.json_normalize(df_itineraries['itineraries'])).drop(columns='itineraries')
    
    # At this point, 'segments' column is a list of dictionaries where each dictionary represents a leg of the journey.
    # We want each leg to be a separate row in the dataframe. So, explode the 'segments' column.
    df_itineraries = df_itineraries.explode('segments').reset_index(drop=True)
    
    # Add a 'leg_id' column to identify each leg of the journey
    df_itineraries['leg_id'] = df_itineraries.groupby('id').cumcount() + 1
    
    # Now, convert the dictionaries in the 'segments' column into separate columns
    df_segments = pd.json_normalize(df_itineraries['segments'])
    
    # To avoid overlapping columns, add a prefix to the column names of the new dataframe
    df_segments.columns = ['flight_' + str(col) for col in df_segments.columns]
    
    # Now join the original dataframe with the new one
    df_itineraries = df_itineraries.join(df_segments).drop(columns='segments')
    
    df_validatingAirlineCodes = df[['id', 'validatingAirlineCodes']]
    
    # For the price column, we only need total and currency. So, extract only those into a new dataframe
    df_price = df['price'].apply(pd.Series)[['total', 'currency']]
    df_price['id'] = df['id']
    
    # Now join these dataframes on the 'id' column
    df_flights = pd.merge(df_itineraries, df_validatingAirlineCodes, on='id')
    df_flights = pd.merge(df_flights, df_price, on='id')
    
    # Create a new column for the total number of legs per journey
    df_flights['total_legs'] = df_flights.groupby('id')['leg_id'].transform('max')
    
    df_flights = df_flights.merge(right=df_airline_codes, how='left', left_on="flight_operating.carrierCode", right_on="iataCode")
    df_flights.rename(columns={"id":"journey_id", "commonName":"airline" }, inplace=True)

    df_flights.drop(columns=["flight_id", "validatingAirlineCodes", "businessName", "flight_operating.carrierCode", "flight_aircraft.code", "flight_stops"], inplace=True)

    df_flights.columns = df_flights.columns.str.replace('.', '_')
    df_flights['total'] = pd.to_numeric(df_flights['total'], errors='coerce')

    outbound_origin = originLocationCode
    outbound_destination = destinationLocationCode
    inbound_origin = destinationLocationCode
    inbound_destination = originLocationCode

    # Create conditions
    cond1 = (df_flights['flight_departure_iataCode'] == outbound_origin) | (df_flights['flight_arrival_iataCode'] == destinationLocationCode)
    cond2 = (df_flights['flight_departure_iataCode'] == inbound_origin) | (df_flights['flight_arrival_iataCode'] == inbound_destination)
    
    # Update 'Journey Start' and 'Journey End' based on conditions
    df_flights.loc[cond1, 'Journey Start'] = originLocationCode
    df_flights.loc[cond1, 'Journey End'] = destinationLocationCode
    df_flights.loc[cond2, 'Journey Start'] = destinationLocationCode
    df_flights.loc[cond2, 'Journey End'] = originLocationCode
    
    # Update 'travel_direction' based on 'Journey Start'
    df_flights.loc[df_flights['Journey Start'] == originLocationCode, 'travel_direction'] = 'Inbound'
    df_flights.loc[df_flights['Journey Start'] == destinationLocationCode, 'travel_direction'] = 'Outbound'


    df_flights.loc[df_flights['flight_arrival_iataCode'] == df_flights['Journey End'], 'flight_arrival_iataCode'] = 'N/A'
    df_flights.loc[df_flights['flight_departure_iataCode'] == df_flights['Journey Start'], 'flight_departure_iataCode'] = 'N/A'

    df_flights.rename(columns={'flight_departure_iataCode': 'intermediate_journey_departure', 
                           'flight_arrival_iataCode': 'intermediate_journey_arrival'}, inplace=True)

    return df_flights

In [13]:
from sqlalchemy import create_engine
import pandas as pd
from langchain import SQLDatabase

def load_data(df_flights):
    engine = create_engine('sqlite:///:memory:')

    # Write the data to the SQLite database
    df_flights.to_sql('flights', engine, if_exists='replace', index=False)
    # Check if the data was loaded correctly
    df_loaded = pd.read_sql('SELECT * FROM flights', engine)
    db = SQLDatabase(engine)
    return db

In [6]:
import json
import openai
from langchain.tools import tool

def get_args(query_user: str) -> str:
    # OpenAI function calling

    """Get's arguments based on client query, 
    returns num_adults, departureDate, returnDate destinationLocationCode, originLocationCode.
    This is required before pulling the data from the API.
    """
    
    function_call = [
    {
      "name": "search_for_flights",
      "description": "Requests flight data from Amadeus API and writes to SQLite database",
      "parameters": {
        "type": "object",
        "properties": {
            "num_adults":{
                "type":"integer",
                "description": '''Based on the query, respond with the number of adults'''
            },
            "departureDate": {
                "type":"string",
                "description": '''Based on the query, respond with the Departure Date. Dates are specified in the ISO 8601 YYYY-MM-DD format. '''
            },
            "returnDate": {
                "type":"string",
                "description": '''Based on the query, respond with the Return Date. Dates are specified in the ISO 8601 YYYY-MM-DD format. '''
            },
            "destinationLocationCode":{
                "type":"string",
                "description": '''Based on the query, respond with an airport IATA code from the city which the traveler is going. E.g CDG for Charles de Gaulle Airport'''
            },
          "originLocationCode": {
            "type": "string",
            "description": '''Based on the query, respond with an airport IATA code from the city which the traveler will depart from. E.g CDG for Charles de Gaulle Airport'''
          },

        },
        "required": ["destinationLocationCode", "originLocationCode", "departureDate", "returnDate", "num_adults"]
      }
    }
    ]
    
    openai.api_key = openai_key

    message = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[{"role": "user", "content": query_user}],
        functions = function_call,
        function_call = 'auto',
        temperature=0
    )
    response_message = message["choices"][0]["message"]["function_call"]["arguments"]

    parsed_data = json.loads(response_message)

    # Accessing variables
    num_adults = parsed_data['num_adults']
    departureDate = parsed_data['departureDate']
    returnDate = parsed_data['returnDate']
    destinationLocationCode = parsed_data['destinationLocationCode']
    originLocationCode = parsed_data['originLocationCode']
    
    print("Number of Adults: ", num_adults)
    print("Departure Date: ", departureDate)
    print("Return Date: ", returnDate)
    print("Destination Location Code: ", destinationLocationCode)
    print("Origin Location Code: ", originLocationCode)

    return num_adults, departureDate, returnDate, destinationLocationCode, originLocationCode

num_adults, departureDate, returnDate, destinationLocationCode, originLocationCode = get_args(query_user)

Number of Adults:  1
Departure Date:  2023-08-30
Return Date:  2023-09-15
Destination Location Code:  HND
Origin Location Code:  LHR


In [70]:
from amadeus import Client, ResponseError
from datetime import datetime
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain.tools import tool
from langchain import LLMMathChain
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

def search_for_flights(originLocationCode, destinationLocationCode, departureDate, returnDate, num_adults) -> SQLDatabase:
    """Requests flight data from Amadeus API and writes to sqllite database and run SQLDatabaseQuery
    originLocationCode: Based on the query, respond with the iataCode for the origin airport,
    destinationLocationCode: Based on the query, respond with the iataCode for the destination airport,
    departureDate: Based on the query, respond with the departure date,
    num_adults: Based on the query, respond with the number of adults
    """

    # Assuming you've defined api_key and api_secret somewhere else
    amadeus = Client(client_id=api_key, client_secret=api_secret)

    # Defining the parameters for the flight
    params = {
        'originLocationCode': originLocationCode,
        'destinationLocationCode': destinationLocationCode,
        'departureDate': departureDate,
        'returnDate': returnDate,
        'adults': num_adults
    }
    
    try:
        response_flights = amadeus.shopping.flight_offers_search.get(**params)
        
    except ResponseError as error:
        print(f"ResponseError occurred flights: {error}")
        print(f"Error code flights: {error.code}")
        print(f"Error message flights: {error.description}")
        return []  # return an empty list in case of an error

    try:
        response_airline_lookup = amadeus.reference_data.airlines.get()

    except ResponseError as error:
        print(f"ResponseError occurred airline lookup: {error}")
        print(f"Error code airline lookup: {error.code}")
        print(f"Error message airline lookup: {error.description}")

    df_flights = journey_data(response_flights.data, response_airline_lookup.data, destinationLocationCode, originLocationCode)
    print(df_flights.dtypes)
    db = load_data(df_flights)

    return db, df_flights

db, df_flights = search_for_flights(originLocationCode, destinationLocationCode, departureDate, returnDate, num_adults)
df_flights.info()

journey_id                    object
duration                      object
leg_id                         int64
flight_carrierCode            object
flight_number                 object
flight_duration               object
flight_numberOfStops           int64
flight_blacklistedInEU          bool
flight_departure_iataCode     object
flight_departure_terminal     object
flight_departure_at           object
flight_arrival_iataCode       object
flight_arrival_terminal       object
flight_arrival_at             object
total                        float64
currency                      object
total_legs                     int64
type                          object
iataCode                      object
icaoCode                      object
airline                       object
Journey Start                 object
Journey End                   object
travel_direction              object
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 24 col

In [8]:
# from langchain.prompts.prompt import PromptTemplate

# _DEFAULT_TEMPLATE = """Based on the user query about flights:{query_user}, respond with the following structure delimited by quotation marks as an example:
    
# "
# The response should be returned like this for the example of a customer flying from London to Tokyo and back: 

# I now have the 5 cheapest Journeys from London to Tokyo departing on the 30th of August 2023 and returning on the 15th of September 2023. 

# | Journey ID   |   Leg ID | Outbound Departure   | Outbound Arrival    | Return Departure    | Return Arrival      | Dep Airport   | Arrival Airport   | Airline    | Total       |
# |:-------------|---------:|:---------------------|:--------------------|:--------------------|:--------------------|:--------------|:------------------|:-----------|:------------|
# | 167.0        |        1 | 2023-08-30T09:40:00  | 2023-08-31T10:40:00 | N/A                 | N/A                 | LHR           | CDG               | AIR France | 1422.79 EUR |
# |              |        2 | 2023-08-31T10:40:00  | 2023-08-31T22:40:00 | N/A                 | N/A                 | CDG           | HND               | AIR France | 1422.79 EUR |
# |              |        1 | N/A                  | N/A                 | 2023-09-15T09:40:00 | 2023-09-16T05:35:00 | HND           | CDG               | AIR France | 1422.79 EUR |
# |              |        2 | N/A                  | N/A                 | 2023-09-16T06:35:00 | 2023-09-16T07:35:00 | CDG           | LHR               | AIR France | 1422.79 EUR |
# | 168.0        |        1 | 2023-08-30T09:40:00  | 2023-08-31T22:40:00 | N/A                 | N/A                 | LHR           | HND               | Air Tokyo  | 1550 EUR    |
# |              |        1 | N/A                  | N/A                 | 2023-09-15T09:40:00 | 2023-09-16T07:35:00 | HND           | LHR               | Air Tokyo  | 1550 EUR    |

# "
# Journeys can have multiple legs as denoted by their leg_id. If you wanted to get the cheapest Journey, do not filter based on the flight destination. All journeys have already been prefiltered.

# For example:
# Tokyo to London could be 2 two legs HND to CDG then CDG to LHR. It could also be HND to CDG. Filtering for departure = CDG and destination = LHR
# will lead you to miss Journeys with two legs. Return ALL leg_id for each Journey ID. Therefore, getting the cheapest journey would be just sorting the table by ascending price.

# when the requests says flight, it really means journey.

# '''"""
# PROMPT = PromptTemplate(
#     input_variables=["query_user"], template=_DEFAULT_TEMPLATE
# )

In [9]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(temperature=0, model="gpt-4-0613", openai_api_key=openai_key)

def find_flights(query, llm):
    '''creates agent that can be run on db to answer query flights'''
    llm=llm
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    agent_executor = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    )
 
    return agent_executor.run(query)


response = find_flights(query, llm)
response



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: flights
Thought:The 'flights' table seems to be the most relevant one for this query. I should check its schema to understand the structure and the fields it contains.
Action: sql_db_schema
Action Input: "flights"
Observation: 
CREATE TABLE flights (
	journey_id TEXT, 
	duration TEXT, 
	leg_id BIGINT, 
	"flight_carrierCode" TEXT, 
	flight_number TEXT, 
	flight_duration TEXT, 
	"flight_numberOfStops" BIGINT, 
	"flight_blacklistedInEU" BOOLEAN, 
	"flight_departure_iataCode" TEXT, 
	flight_departure_terminal TEXT, 
	flight_departure_at TEXT, 
	"flight_arrival_iataCode" TEXT, 
	flight_arrival_terminal TEXT, 
	flight_arrival_at TEXT, 
	total FLOAT, 
	currency TEXT, 
	total_legs BIGINT, 
	type TEXT, 
	"iataCode" TEXT, 
	"icaoCode" TEXT, 
	airline TEXT, 
	"Journey Start" TEXT, 
	"Journey End" TEXT
)

/*
3 rows from flights table:
journey_id	duration	leg_id	flight_carrierCode	flight_number	flight_d

'Here are the 10 cheapest flights from London to Tokyo departing on or after 30 August 2023 and arriving on or before 15 September 2023:\n\n1. Journey ID: 170, Duration: PT11H55M, Leg ID: 1, Carrier Code: JL, Flight Number: 42, Flight Duration: PT11H55M, Number of Stops: 0, Departure: LHR at 2023-08-30T09:40:00, Arrival: HND at 2023-08-31T05:35:00, Total Price: 1422.79, Total Legs: 2\n2. Journey ID: 226, Duration: PT11H55M, Leg ID: 1, Carrier Code: JL, Flight Number: 42, Flight Duration: PT11H55M, Number of Stops: 0, Departure: LHR at 2023-08-30T09:40:00, Arrival: HND at 2023-08-31T05:35:00, Total Price: 1487.79, Total Legs: 2\n3. Journey ID: 227, Duration: PT11H55M, Leg ID: 1, Carrier Code: JL, Flight Number: 44, Flight Duration: PT11H55M, Number of Stops: 0, Departure: LHR at 2023-08-30T19:20:00, Arrival: HND at 2023-08-31T15:15:00, Total Price: 1487.79, Total Legs: 2\n4. Journey ID: 228, Duration: PT11H55M, Leg ID: 1, Carrier Code: JL, Flight Number: 44, Flight Duration: PT11H55M, N

In [71]:
df_flights[["journey_id", "flight_departure_iataCode", "flight_arrival_iataCode", "Journey Start", "Journey End", "travel_direction"]].loc[df_flights['journey_id'] == '180']

,journey_id,flight_departure_iataCode,flight_arrival_iataCode,Journey Start,Journey End,travel_direction
716,180,LHR,HKG,LHR,HND,Outbound
717,180,HKG,HND,LHR,HND,Outbound
718,180,HND,HKG,HND,LHR,Inbound
719,180,HKG,LHR,HND,LHR,Inbound


In [ ]:
# Create conditions
cond1 = (df_flights['flight_departure_iataCode'] == outbound_origin) | (df_flights['flight_arrival_iataCode'] == destinationLocationCode)
cond2 = (df_flights['flight_departure_iataCode'] == inbound_origin) | (df_flights['flight_arrival_iataCode'] == inbound_destination)

# Update 'Journey Start' and 'Journey End' based on conditions
df_flights.loc[cond1, 'Journey Start'] = originLocationCode
df_flights.loc[cond1, 'Journey End'] = destinationLocationCode
df_flights.loc[cond2, 'Journey Start'] = destinationLocationCode
df_flights.loc[cond2, 'Journey End'] = originLocationCode

# Update 'travel_direction' based on 'Journey Start'
df_flights.loc[df_flights['Journey Start'] == originLocationCode, 'travel_direction'] = 'Inbound'
df_flights.loc[df_flights['Journey Start'] == destinationLocationCode, 'travel_direction'] = 'Outbound'


In [11]:
df_flights.sort_values(by='total', ascending=True)

flight_arrival_iataCode	
flight_departure_iataCode

NameError: name 'flight_arrival_iataCode' is not defined

In [ ]:
outbound_origin = originLocationCode
outbound_destination = destinationLocationCode
inbound_origin = destinationLocationCode
inbound_destination = originLocationCode

df_flights['Journey Start'] = originLocationCode
df_flights['Journey End'] = destinationLocationCode
df_flights['travel_direction'] = np.where(df_flights['flight_departure_iataCode'] == originLocationCode, 'Outbound', 'Inbound')


In [ ]:
df_flights.dtypes

In [ ]:
# import pandas as pd
# import numpy as np

# if flight_departure_iataCode equal originLocationCode or flight_arrival_iataCode not equal destinationLocationCode
# df_flights['Journey Start'] = originLocationCode
# df_flights['Journey End'] = destinationLocationCode
# if flight_departure_iataCode equal destinationLocationCode or flight_arrival_iataCode not equal originLocationCode
# df_flights['Journey Start'] = destinationLocationCode
# df_flights['Journey End'] = originLocationCode

# if df_flights['Journey Start'] = originLocationCode then df_fights['travel_direction'] = outbound
# if df_flights['Journey Start'] = destinationLocationCode then df_fights['travel_direction'] = Inbound


# # Replace flight_arrival_iataCode and flight_departure_iataCode with 'N/A' based on conditions
# df_flights.loc[df_flights['flight_arrival_iataCode'] == df_flights['Journey End'], 'flight_arrival_iataCode'] = 'N/A'
# df_flights.loc[df_flights['flight_departure_iataCode'] == df_flights['Journey Start'], 'flight_departure_iataCode'] = 'N/A'

# # Rename the columns
# df_flights.rename(columns={'flight_departure_iataCode': 'intermediate_journey_departure', 
#                            'flight_arrival_iataCode': 'intermediate_journey_arrival'}, inplace=True)
